In [ ]:
# Branch-and-Bound para coin-collecting en una cuadrícula n x n.

# Complejidad peor caso: O(n^2)

# Upper-bound:
# - Para una solución parcial en posición (r,c) con suma actual S, quedan k =
#   (n-1 - r) + (n-1 - c) movimientos (cada uno entra a una nueva celda dentro de la
#   submatriz [r..n-1] x [c..n-1]).
#   Tomamos los k valores más grandes dentro de la submatriz de celdas no
#   visitadas (a partir de (r,c) inclusive, pero excluyendo la celda actual si ya
#   fue contada) y los sumamos. Cualquier camino factible solo podrá recoger a lo sumo
#   esa cantidad adicional, por lo que es una cota superior válida y razonablemente
#   ajustada.

import sys
import heapq

class Node:
    def __init__(self, r, c, moves, sum_collected):
        self.r = r
        self.c = c
        self.moves = moves  # list of 'D' or 'R'
        self.sum = sum_collected
        self.ub = sum_collected

    def __lt__(self, other):
        return self.ub < other.ub

def read_instance(path):
    with open(path, 'r', encoding='utf-8') as f:
        lines = [ln.strip() for ln in f if ln.strip()!='']
    n = int(lines[0])
    grid = []
    for i in range(1, 1+n):
        # acepta tabs o espacios
        parts = lines[i].replace('\t', ' ').split()
        grid.append([int(x) for x in parts])
    return n, grid

def upper_bound_for(node, grid, n):
    """
    Calcula UB para un nodo parcial en (r,c) con suma actual node.sum.
    UB = node.sum + suma de los k mayores valores en la submatriz [r..n-1][c..n-1],
    excluyendo la celda actual (porque ya fue agregada en node.sum).
    """
    r, c = node.r, node.c
    remaining_steps = (n-1 - r) + (n-1 - c)
    if remaining_steps <= 0:
        return node.sum
    vals = []
    for i in range(r, n):
        for j in range(c, n):
            if i==r and j==c:
                continue  # celda actual ya contada en node.sum
            vals.append(grid[i][j])
    vals.sort(reverse=True)
    k = min(remaining_steps, len(vals))
    optimistic_add = sum(vals[:k])
    return node.sum + optimistic_add

def expand(node, grid, n):
    children = []
    # Move Right
    if node.c + 1 < n:
        nr, nc = node.r, node.c + 1
        val = grid[nr][nc]
        ch = Node(nr, nc, node.moves + ['R'], node.sum + val)
        children.append(ch)
    # Move Down
    if node.r + 1 < n:
        nr, nc = node.r + 1, node.c
        val = grid[nr][nc]
        ch = Node(nr, nc, node.moves + ['D'], node.sum + val)
        children.append(ch)
    return children

def branch_and_bound(n, grid):
    # Nodo raíz (estamos en (0,0) y hemos recogido su valor)
    root = Node(0,0,[], grid[0][0])
    root.ub = upper_bound_for(root, grid, n)
    # Max-heap emulado con heapq usando (-ub, -sum, counter, node) para desempate
    heap = []
    counter = 0
    heapq.heappush(heap, (-root.ub, -root.sum, counter, root))
    counter += 1

    best_sum = root.sum if (n>0) else 0
    best_node = root if n>0 else None

    while heap:
        neg_ub, neg_sum, _, node = heapq.heappop(heap)
        ub = -neg_ub
        # Poda: si la cota superior no mejora la mejor solución encontrada, ignorar
        if ub <= best_sum:
            continue
        # Si nodo completo (llegó a (n-1,n-1))
        if node.r == n-1 and node.c == n-1:
            if node.sum > best_sum:
                best_sum = node.sum
                best_node = node
            continue
        # Expandir
        for ch in expand(node, grid, n):
            ch.ub = upper_bound_for(ch, grid, n)
            # Poda por suma parcial si la suma parcial ya es mejor que best, we keep
            if ch.ub <= best_sum:
                continue
            heapq.heappush(heap, (-ch.ub, -ch.sum, counter, ch))
            counter += 1

    return best_node, best_sum

def format_solution(node, grid):
    # Construccion de la cadena
    if node is None:
        return "No hay solución", 0
    moves = node.moves
    r, c = 0, 0
    parts = []
    for mv in moves:
        if mv == 'R':
            c += 1
            parts.append(f"derecha({grid[r][c]})")
        else:
            r += 1
            parts.append(f"abajo({grid[r][c]})")
    total = node.sum
    if parts:
        return ', '.join(parts) + f". Total={total}", total
    else:
        # sin movimientos (n==1)
        return f"Total={grid[0][0]}", grid[0][0]

def main():
    path = 'coins-n12.txt'
    n, grid = read_instance(path)
    best_node, best_sum = branch_and_bound(n, grid)
    sol_str, total = format_solution(best_node, grid)
    print(sol_str)

if __name__ == '__main__':
    main()

abajo(2), derecha(3), abajo(2), abajo(4), abajo(2), derecha(3), abajo(3), derecha(4), abajo(2), derecha(0), derecha(4), derecha(3), abajo(4), abajo(4), derecha(4), derecha(0), derecha(2), abajo(4), derecha(0), abajo(3), derecha(3), abajo(1). Total=57
